In [6]:
from datetime import datetime,timedelta

In [16]:
exec_datetime = '2019-12-07 07:21:32'
wrf_run = '0'
wrf_run = int(wrf_run)
gfs_hour = '06'
exec_datetime = datetime.strptime(exec_datetime, '%Y-%m-%d %H:%M:%S')
print(exec_datetime)
exec_date_str = exec_datetime.strftime('%Y-%m-%d')
exec_date = datetime.strptime(exec_date_str, '%Y-%m-%d')
print(exec_date)

2019-12-07 07:21:32
2019-12-07 00:00:00


In [17]:
ts_start_date = exec_date - timedelta(days=wrf_run)
ts_start_date_str = ts_start_date.strftime('%Y-%m-%d')
print(ts_start_date_str)

2019-12-07


In [19]:
gfs_ts_start_utc_str = '{} {}:00:00'.format(ts_start_date_str, gfs_hour)
print(gfs_ts_start_utc_str)

2019-12-07 06:00:00


In [20]:
gfs_ts_start_utc = datetime.strptime(gfs_ts_start_utc_str, '%Y-%m-%d %H:%M:%S')
gfs_ts_start_local = gfs_ts_start_utc + timedelta(hours=5, minutes=30)
print(gfs_ts_start_local)

2019-12-07 11:30:00


In [21]:
gfs_ts_end_local = gfs_ts_start_local + timedelta(days=3)
print(gfs_ts_end_local)

2019-12-10 11:30:00


In [4]:
station_accuracy = '60'
obs_stations = 'IBATTARA2-62, Kottawa North Dharmapala School-55,Kohuwala'
station_list = obs_stations.split(",")
print(station_list)
formatted_list = []
for station in station_list:
    station_val = station.split('-')
    if len(station_val)==2:
        formatted_list.append([station_val[0],station_val[1]])
    else:
        formatted_list.append([station_val[0],station_accuracy])
print(formatted_list)

['IBATTARA2-62', ' Kottawa North Dharmapala School-55', 'Kohuwala']
[['IBATTARA2', '62'], [' Kottawa North Dharmapala School', '55'], ['Kohuwala', '60']]


In [7]:
import sys
sys.path.insert(0, '/home/hasitha/PycharmProjects/DSS-Framework/db_util')
from gen_db import CurwFcstAdapter, CurwObsAdapter, CurwSimAdapter
from dss_db import RuleEngineAdapter

def get_curw_dss_adapter(db_config=None):
    if db_config is None:
        db_config = Variable.get('db_config', deserialize_json=True)
    adapter = RuleEngineAdapter.get_instance(db_config)
    return adapter


def get_curw_fcst_adapter(db_config=None):
    if db_config is None:
        db_config = Variable.get('fcst_db_config', deserialize_json=True)
    adapter = CurwFcstAdapter.get_instance(db_config)
    return adapter


def get_curw_obs_adapter(db_config=None):
    if db_config is None:
        db_config = Variable.get('obs_db_config', deserialize_json=True)
    adapter = CurwObsAdapter.get_instance(db_config)
    return adapter


def get_curw_sim_adapter(db_config=None):
    if db_config is None:
        db_config = Variable.get('sim_db_config', deserialize_json=True)
    adapter = CurwSimAdapter.get_instance(db_config)
    return adapter

obs_db_config = {'mysql_user': 'admin', 'mysql_password': 'floody', 'mysql_host': '35.227.163.211',
               'mysql_db': 'curw_obs', 'log_path': '/home/hasitha/PycharmProjects/DSS-Framework/log'}
sim_db_config = {'mysql_user': 'admin', 'mysql_password': 'floody', 'mysql_host': '35.227.163.211',
                  'mysql_db': 'curw_sim', 'log_path': '/home/hasitha/PycharmProjects/DSS-Framework/log'}
fcst_db_config = {'mysql_user': 'admin', 'mysql_password': 'floody', 'mysql_host': '35.227.163.211',
                'mysql_db': 'curw_fcst', 'log_path': '/home/hasitha/PycharmProjects/DSS-Framework/log'}
dss_db_config = {'mysql_user': 'admin', 'mysql_password': 'floody', 'mysql_host': '35.227.163.211',
                'mysql_db': 'dss', 'log_path': '/home/hasitha/PycharmProjects/DSS-Framework/log'}

dss_adapter = get_curw_dss_adapter(dss_db_config)
fcst_adapter = get_curw_fcst_adapter(fcst_db_config)
obs_adapter = get_curw_obs_adapter(obs_db_config)
sim_adapter = get_curw_sim_adapter(sim_db_config)

get_instance|db_config :  {'mysql_user': 'admin', 'mysql_password': 'floody', 'mysql_host': '35.227.163.211', 'mysql_db': 'curw_fcst', 'log_path': '/home/hasitha/PycharmProjects/DSS-Framework/log'}
get_instance|db_config :  {'mysql_user': 'admin', 'mysql_password': 'floody', 'mysql_host': '35.227.163.211', 'mysql_db': 'curw_obs', 'log_path': '/home/hasitha/PycharmProjects/DSS-Framework/log'}
get_instance|db_config :  {'mysql_user': 'admin', 'mysql_password': 'floody', 'mysql_host': '35.227.163.211', 'mysql_db': 'curw_sim', 'log_path': '/home/hasitha/PycharmProjects/DSS-Framework/log'}


In [8]:
def calculate_wrf_rule_accuracy(wrf_rule, exec_datetime, dss_adapter):
    print('calculate_wrf_rule_accuracy|wrf_rule : ', wrf_rule)
    print('calculate_wrf_rule_accuracy|execution_date : ', exec_datetime)
    wrf_model = 'WRF_{}'.format(wrf_rule['model'])
    print('calculate_wrf_rule_accuracy|wrf_model : ', wrf_model)
    wrf_version = wrf_rule['version']
    wrf_run = wrf_rule['rule_info']['run']
    gfs_hour = wrf_rule['rule_info']['hour']
    accuracy_rule_id = wrf_rule['rule_info']['accuracy_rule']
    sim_tag = 'gfs_d{}_{}'.format(wrf_run, gfs_hour)
    print('calculate_wrf_rule_accuracy|sim_tag : ', sim_tag)
    accuracy_rule = dss_adapter.get_accuracy_rule_info_by_id(accuracy_rule_id)
    print('calculate_wrf_rule_accuracy|accuracy_rule : ', accuracy_rule)
    obs_station_list = format_obs_station_list(accuracy_rule['observed_stations'], accuracy_rule['station_accuracy'])
    if len(obs_station_list) > 0:
        for [obs_station, accuracy_level] in obs_station_list:
            calculate_station_accuracy(obs_station, wrf_model, wrf_version, wrf_run, gfs_hour, exec_datetime, sim_tag)